In [3]:
!pip install xmltodict
!pip install --upgrade --quiet  google-search-results
!pip install langchainhub

In [61]:
import os
import openai

from dotenv import load_dotenv
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
from langchain import hub
from langchain.prompts import PromptTemplate

# Setting Up 

In [5]:
#loading environment variables 
load_dotenv("api.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV') 

#SERP API necessary for Google Scholar API 
SERP_API_KEY = os.getenv("SERP_API_KEY")

# PubMed Tool

In [6]:
#Setting up PubMed Tool 
pubmed_tool = PubmedQueryRun()

In [7]:
#Output when PubMed Tool is run 
question = "How effective are peer support groups, facilitated through social media in helping women with PCOS?"
pubmed_tool.invoke(question)

'No good PubMed Result was found'

In [8]:
#Output when PubMed Tool is run 
question = "What are the effects of caffeine and alcohol intake on PCOS symptoms?"
pubmed_tool.invoke(question)

'Published: --\nTitle: Lifestyle choices, diet, and insulin sensitizers in polycystic ovary syndrome.\nCopyright Information: \nSummary::\nPolycystic ovary syndrome (PCOS) is significantly affected by environmental regulators impacting on genetic predisposition. Lifestyle changes can significantly modulate the phenotype of this disease. Diet, exercise, smoking, stress, and other factors adversely affect reproductive outcomes in PCOS. These influences can be modulated by structure change in an individual or group. Lifestyle choices should be discussed in this group of patients. The role of insulin sensitizers, including metformin, has still to be determined in this condition.'

# Google Scholar Tool 

In [9]:
#Setting up Google Scholar Tool 
api_wrapper = GoogleScholarAPIWrapper(top_k_results = 1)
google_tool = GoogleScholarQueryRun(api_wrapper=api_wrapper)

In [10]:
#Output when Google tool is run 
question = "How effective are peer support groups, facilitated through social media in helping women with PCOS?"
google_tool.run(question)

'Title: Nurse‐led peer support group: Experiences of women with polycystic ovary syndrome\nAuthors: CA Percy,L Potter\nSummary: CA Percy, T Gibbs, L Potter… - Journal of Advanced …, 2009 - Wiley Online Library\nTotal-Citations: 85'

In [11]:
#Output when Google tool is run 
question = "What are the effects of caffeine and alcohol intake on PCOS symptoms?"
google_tool.run(question)

'Title: Alcohol, drugs, caffeine, tobacco, and environmental contaminant exposure: reproductive health consequences and clinical implications\nAuthors: CL Hughes\nSummary: JC Sadeu, CL Hughes, S Agarwal… - Critical reviews in …, 2010 - Taylor & Francis\nTotal-Citations: 154'

# Guideline Tool 

In [12]:
#Loading and splitting documents
loader = TextLoader("Guidelines (txt)/master_pcos.txt")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=200).split_documents(docs)

In [13]:
#Embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

In [14]:
#Retriever
index_name = "tools"
pinecone = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)
retriever = pinecone.as_retriever()

In [15]:
#Creating guideline tool -> named retriever tool 
retriever_tool = create_retriever_tool(retriever,"pcos_search",
                                      "You are a PCOS chatbot. Answer the question given the context.")

In [16]:
retriever_tool.name

'pcos_search'

# RAG

In [17]:
#Set LLM model 
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0)

In [32]:
#LangChain toolkit -> order = in what order they are used! 
tools=[retriever_tool,pubmed_tool,google_tool]

In [33]:
#Prompt 
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [44]:
# Agents
agent=create_openai_tools_agent(model,tools,prompt)

In [38]:
#Agent Executor 
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=False, handle_parsing_errors = True)

# Evaluation

In [46]:
query = "What specific types of exercise are recommended for women with PCOS?"
agent_executor.invoke({"input":query})

{'input': 'What specific types of exercise are recommended for women with PCOS?',
 'output': 'For women with PCOS, it is recommended to engage in aerobic activity in bouts of at least 10 minutes duration, aiming for at least 30 minutes daily on most days. Additionally, barriers and facilitators to engagement and adherence to physical activity should be discussed, including psychological factors, personal safety concerns, environmental factors, and personal motivators for change. Referral to allied healthcare professionals may be considered to optimize physical activity in women with PCOS.\n\nSelf-monitoring with fitness tracking devices and technologies for step count and exercise intensity could be beneficial to support and promote active lifestyles and reduce sedentary behaviors. High-intensity interval training and moderate-intensity continuous training are both effective for improving metabolic and hormonal outcomes in women with PCOS.\n\nOverall, lifestyle interventions such as ex

In [40]:
query = "What anti-androgen medications are recommended for treating hirsutism associated with PCOS?"
agent_executor.invoke({"input":query})

{'input': 'What anti-androgen medications are recommended for treating hirsutism associated with PCOS?',
 'output': "The recommendations for treating hirsutism associated with PCOS include considering anti-androgen medications in combination with effective contraception if there is a suboptimal response to other therapies like COCP and cosmetic therapy. Some anti-androgen medications that could be considered are spironolactone, cyproterone acetate, finasteride, flutamide, and bicalutamide. It's important to note that spironolactone at 25-100 mg/day appears to have lower risks of adverse effects compared to other medications. However, healthcare professionals should be cautious about the potential risks and side effects associated with each medication.\n\nAdditionally, it is recommended to educate and counsel women about the risks of using anti-androgens, especially in women who can get pregnant, to prevent incomplete development of external genital structures in male fetuses. Effective

In [41]:
query = "Discuss the impact of PCOS on psychosexual function and the recommended therapeutic interventions."
agent_executor.invoke({"input":query})

{'input': 'Discuss the impact of PCOS on psychosexual function and the recommended therapeutic interventions.',
 'output': 'Psychosexual dysfunction in women with Polycystic Ovary Syndrome (PCOS) can be influenced by various factors such as depression, low self-esteem, and negative body image. Studies have shown that women with PCOS may experience lower sexual function, arousal, lubrication, orgasm, and satisfaction compared to those without PCOS. However, there were no significant differences in sexual desire or pain between the two groups.\n\nTherapeutic interventions for addressing psychosexual dysfunction in PCOS may include considering factors like higher weight, hirsutism, mood disorders, infertility, and medications used for PCOS treatment. Healthcare professionals should be mindful of these factors when discussing psychosexual function with individuals with PCOS.\n\nIt is important to seek permission to discuss psychosexual function with patients and consider both low psychosex

In [42]:
query = "What are the potential impacts of early life environmental exposures (e.g., endocrine disruptors) on the onset and progression of PCOS symptoms throughout life stages?"
agent_executor.invoke({"input":query})

{'input': 'What are the potential impacts of early life environmental exposures (e.g., endocrine disruptors) on the onset and progression of PCOS symptoms throughout life stages?',
 'output': 'Early life environmental exposures, such as endocrine disruptors, can have potential impacts on the onset and progression of PCOS symptoms throughout life stages. Here are some key points based on the search results:\n\n1. **Pregnancy Outcomes**: Women with PCOS are at a higher risk of adverse pregnancy outcomes due to comorbidities like high BMI, metabolic disturbances, hyperandrogenism, and infertility. However, the detection of PCOS symptoms during pregnancy is challenging, leading to a lack of awareness and management of the condition during antenatal care and postpartum.\n\n2. **Preconception Risk Factors**: Preconception care for women with PCOS is crucial to identify and modify risk factors that can impact fertility and pregnancy outcomes. Factors such as BMI, smoking, alcohol consumption,

In [47]:
query = "Are there any specific proteomic or metabolomic profiles that could redefine the risk assessment of PCOS in the near future?"
agent_executor.invoke({"input":query})

{'input': 'Are there any specific proteomic or metabolomic profiles that could redefine the risk assessment of PCOS in the near future?',
 'output': 'Specific proteomic or metabolomic profiles that could redefine the risk assessment of PCOS in the near future include:\n\n1. Biochemical Hyperandrogenism:\n   - Hyperandrogenism is a key diagnostic feature of PCOS affecting a significant percentage of individuals with the condition.\n   - Assessment of biochemical hyperandrogenism is challenging due to variations in methods, ethnicity, and confounding factors.\n   - Measures of laboratory-calculated unbound or free testosterone are commonly used for assessment.\n   - High-quality assays for testosterone measurements are essential for accurate diagnosis.\n\n2. Clinical Hyperandrogenism:\n   - PCOS is associated with a higher prevalence of cardiometabolic risk factors, including weight issues, impaired glucose tolerance, diabetes, dyslipidemia, and metabolic syndrome.\n   - Women with PCOS 

In [48]:
query = "Is there a potential for non-invasive imaging technologies to replace current diagnostic criteria involving hormonal assessments and ultrasounds in the future?"
agent_executor.invoke({"input":query})

{'input': 'Is there a potential for non-invasive imaging technologies to replace current diagnostic criteria involving hormonal assessments and ultrasounds in the future?',
 'output': 'Non-invasive imaging technologies have the potential to play a significant role in the diagnosis of Polycystic Ovary Syndrome (PCOS) by potentially replacing current diagnostic criteria involving hormonal assessments and ultrasounds. Here are some key points related to this topic:\n\n1. **Ultrasound and Polycystic Ovarian Morphology (PCOM):**\n   - Ultrasound criteria for diagnosing PCOS may not be suitable for adolescents with a gynecological age of less than 8 years due to limitations in peak ovarian maturity, rising anti-mullerian levels, and the high incidence of multi-follicular ovaries.\n   - In adults with features of hyperandrogenism and ovulatory dysfunction meeting PCOS diagnostic criteria, ultrasound may not be necessary for diagnosis.\n   - Technology advancements in the last decade support a

In [49]:
query = "How effective are peer support groups, facilitated through social media in helping women with PCOS?"
agent_executor.invoke({"input":query})

{'input': 'How effective are peer support groups, facilitated through social media in helping women with PCOS?',
 'output': 'Peer support groups facilitated through social media can be effective in helping women with PCOS by providing a platform for social and peer support. These groups can reduce stress, increase confidence, and provide a sense of security for women with PCOS. Additionally, peer support can play a role in improving mental health outcomes such as anxiety and depression commonly associated with PCOS. It is important to note that peer support groups should be complemented with other interventions like psychological therapy and exercise interventions for a comprehensive approach to managing PCOS. \n\nIf you need more detailed information or specific studies on the effectiveness of peer support groups facilitated through social media for women with PCOS, please let me know.'}

In [50]:
query = "Are there any specific gene-environment interactions that influence the psychological symptoms in PCOS?"
agent_executor.invoke({"input":query})

{'input': 'Are there any specific gene-environment interactions that influence the psychological symptoms in PCOS?',
 'output': 'The search results indicate that women with PCOS have a higher prevalence of clinically significant symptoms of depression, anxiety, and eating disorders. These psychological symptoms can impact quality of life and relationships. Hormonal medications used in PCOS treatment can also influence mood. Psychosexual dysfunction, which can be influenced by psychological factors like depression and negative body image, is a common issue in PCOS.\n\nEffective treatments such as cognitive behavioral therapy (CBT), behavioral therapy, and interpersonal therapy are available for managing these psychological symptoms. Additionally, social and family support can play a crucial role in managing PCOS-related psychological challenges.\n\nFurther details and specific gene-environment interactions influencing psychological symptoms in PCOS may require more targeted research or 

In [51]:
query = "What is the potential for using artificial intelligence in screening and managing psychological features in PCOS? "
agent_executor.invoke({"input":query})

{'input': 'What is the potential for using artificial intelligence in screening and managing psychological features in PCOS? ',
 'output': 'Artificial intelligence (AI) has the potential to play a significant role in screening and managing psychological features in Polycystic Ovary Syndrome (PCOS). Here are some key points based on the search results:\n\n1. **Psychological Therapy**: Women with PCOS have a higher prevalence of symptoms like depression, anxiety, and eating disorders. Treatments such as cognitive behavioral therapy (CBT) and behavioral therapy have shown improvements in mental health conditions. Telehealth and online programs with therapist assistance have also been effective.\n\n2. **Screening and Recommendations**: Healthcare professionals should be aware of the high prevalence of depressive and anxiety symptoms in individuals with PCOS. Screening for depression and anxiety using validated tools is recommended. If moderate or severe symptoms are detected, appropriate a

In [52]:
query = "What are the effects of caffeine and alcohol intake on PCOS symptoms?"
agent_executor.invoke({"input":query})

{'input': 'What are the effects of caffeine and alcohol intake on PCOS symptoms?',
 'output': 'The effects of caffeine intake on PCOS symptoms include a higher intake of saturated fats, monounsaturated fats, oleic acid, fluorine, sucrose, and caffeine itself, and a lower intake of vitamins A, B, B, B, C, and D, potassium, proteins, carbohydrates, cholesterols, docosahexaenoic acid, potassium, carotenes, lutein, beta-cryptoxanthin, lycopene, calcium, iron, thiamine, riboflavin, niacin, tetra- and dihydrofolate, biotin, phosphorus, magnesium, zinc, copper, and fiber.\n\nThe effects of alcohol intake on PCOS symptoms were also studied, and the results suggest that alcohol intake frequency may have an impact on PCOS features. However, after adjusting for factors like BMI, SHBG, fasting insulin, and testosterone, the effect sizes were significantly reduced. The study did not find strong evidence supporting a causal relationship between gut microbiota and PCOS features at the genetic level.\

In [53]:
query = "Are there specific micronutrients or supplements that should be considered in the diet of women with PCOS?"
agent_executor.invoke({"input":query})

{'input': 'Are there specific micronutrients or supplements that should be considered in the diet of women with PCOS?',
 'output': "For women with PCOS, here are some recommendations regarding diet, exercise interventions, and supplements based on the search results:\n\n### Diet Recommendations:\n- There is no evidence to support any one type of diet composition over another for women with PCOS.\n- Any diet consistent with population guidelines for healthy eating will have health benefits.\n- Healthcare professionals should advise sustainable healthy eating tailored to individual preferences and goals.\n- Tailoring dietary changes to food preferences and avoiding unduly restrictive diets is important.\n- Barriers and facilitators to optimize engagement and adherence to dietary changes should be discussed.\n- Referral to allied healthcare professionals may be considered for support with optimizing the diet.\n\n### Exercise Interventions:\n- Inositol (in any form) could be considered in 

In [54]:
query = "What are the emerging trends in nutrigenomics that could impact dietary recommendations for PCOS?"
agent_executor.invoke({"input":query})

{'input': 'What are the emerging trends in nutrigenomics that could impact dietary recommendations for PCOS?',
 'output': 'The emerging trends in nutrigenomics that could impact dietary recommendations for PCOS include the following key points:\n\n1. **Dietary Recommendations**:\n   - Diets with a range of macronutrient compositions could be recommended for women with PCOS.\n   - Quality of carbohydrates in the diet is important, with a focus on high-fiber, lower glycemic index carbohydrate foods like whole grains, pulses/legumes, and fruits.\n   - There is no evidence to support any one type of diet composition over another for various outcomes.\n   - Healthcare professionals should advise sustainable healthy eating tailored to individual preferences and goals.\n\n2. **Exercise Interventions**:\n   - Prevention of weight gain is vital for women with PCOS, given the challenges with weight loss and maintenance.\n   - Prioritizing exercise interventions can help in managing weight and im

In [57]:
query = "Are there any novel biomarkers under investigation that could improve the management of dermatologic features in PCOS?"
agent_executor.invoke({"input":query})

{'input': 'Are there any novel biomarkers under investigation that could improve the management of dermatologic features in PCOS?',
 'output': 'Studies have focused on clinical and biochemical markers of hyperandrogenism in PCOS, which impact dermatologic features like hirsutism, acne, and female pattern hair loss. Clinical hyperandrogenism, assessed through tools like the Ferriman Gallwey score, is crucial for diagnosis and treatment. Hirsutism alone can predict biochemical hyperandrogenism and PCOS in adults. Biochemical hyperandrogenism, challenging to assess due to methodological variations, is a key diagnostic feature affecting 60-100% of individuals with PCOS.\n\nRegarding novel biomarkers, the research emphasizes the importance of accurate diagnosis through clinical and biochemical assessments rather than specific novel biomarkers. The presence of hirsutism is a strong predictor of PCOS, while female pattern hair loss and acne alone are weaker indicators. Biochemical hyperandrog

In [58]:
query = "Are there any ongoing clinical trials focusing on immune-modulating treatments for skin manifestations of PCOS? "
agent_executor.invoke({"input":query})

{'input': 'Are there any ongoing clinical trials focusing on immune-modulating treatments for skin manifestations of PCOS? ',
 'output': "I couldn't find specific ongoing clinical trials focusing on immune-modulating treatments for skin manifestations of PCOS. However, I found information on the use of anti-androgen pharmacological agents and recommendations for managing skin manifestations of PCOS. Here are some key points:\n\n- Anti-androgen pharmacological agents could be considered to treat hirsutism in women with PCOS if there is a suboptimal response to other therapies.\n- Anti-androgens in combination with COCP could be trialed for female pattern hair loss in PCOS, despite limited evidence in this population.\n- Healthcare professionals should educate women about the risks of using anti-androgens during pregnancy to prevent adverse effects on male fetuses.\n- Laser and light therapies may be beneficial for reducing facial hirsutism and improving quality of life in women with PCO

In [59]:
query = "What specific genetic polymorphisms in PCOS patients are linked to infertility, and how can targeted therapies be developed based on these findings?"
agent_executor.invoke({"input":query})

{'input': 'What specific genetic polymorphisms in PCOS patients are linked to infertility, and how can targeted therapies be developed based on these findings?',
 'output': 'The search results indicate that women with PCOS have significantly higher odds of experiencing infertility-related complications such as miscarriage, gestational diabetes, gestational hypertension, pre-eclampsia, preterm birth, low birth weight, intrauterine growth restriction, and caesarean section. These outcomes are influenced by factors like BMI, metabolic disturbances, and hyperandrogenism.\n\nSpecific genetic polymorphisms in PCOS patients linked to infertility are not explicitly mentioned in the search results. However, it is known that PCOS is a complex disorder with a genetic component, and several genetic variants have been associated with PCOS and its related symptoms, including infertility.\n\nTo develop targeted therapies based on these findings, further research is needed to identify the specific gen

In [60]:
query = "What role do stem cells play in potential future treatments for infertility in PCOS patients?"
agent_executor.invoke({"input":query})

{'input': 'What role do stem cells play in potential future treatments for infertility in PCOS patients?',
 'output': 'Stem cells have not been specifically mentioned in the search results related to the role of stem cells in potential future treatments for infertility in PCOS patients. However, the search results provide information on assisted reproductive technology (ART) therapies, including in vitro fertilization (IVF) and in vitro maturation (IVM), as potential treatment options for infertility in PCOS patients.\n\nIVF and ICSI could be considered in women with PCOS as an alternative to a stimulated IVF/ICSI cycle, where an embryo is frozen and replaced in a subsequent embryo transfer cycle. This approach acknowledges the lower risk of ovarian hyperstimulation syndrome but may result in a lower cumulative live birth rate. In the absence of an absolute indication for IVF/ICSI, IVF could be offered to women with PCOS and anovulatory infertility if first- or second-line ovulation in